In [ ]:
from bs4 import BeautifulSoup
import requests
import re

def fetch_dining_hall_urls(main_url):
    response = requests.get(main_url, verify=False)
    soup = BeautifulSoup(response.text, 'html.parser')
    dining_hall_links = soup.select('.submenu li a')
    return [link['href'] for link in dining_hall_links if 'select-access' not in link['href']]

def convert_percentage(percentage_str):
    return percentage_str.rstrip('%')

def strip_redundant_text(text, redundant_phrase):
    return text.replace(redundant_phrase, '').strip()

def extract_serving_size_and_calories(nutrition_table):
    serving_size_row = nutrition_table.find('tr', class_='serving-size')
    serving_size = strip_redundant_text(serving_size_row.get_text(strip=True), "Serving Size") if serving_size_row else "N/A"

    calories_row = nutrition_table.find('tr', class_='portion-calories')
    calories = strip_redundant_text(calories_row.get_text(strip=True), "Calories") if calories_row else "N/A"

    return serving_size, calories

def split_nutrient_data(nutrient_str):
    match = re.match(r"([a-zA-Z\s]+)\s*(\d*)", nutrient_str)
    if match:
        return match.group(1).strip(), match.group(2)
    else:
        return nutrient_str, None

main_url = 'https://dining.umich.edu/menus-locations/dining-halls/'
dining_hall_urls = fetch_dining_hall_urls(main_url)

all_dining_data = {}

for url in dining_hall_urls:
    page = requests.get(url, verify=False)
    soup = BeautifulSoup(page.text, 'html.parser')
    courses_wrappers = soup.find_all('ul', class_='courses_wrapper')

    dining_hall_name = url.split('/')[-2]  # Extract dining hall name from URL

    all_dining_data[dining_hall_name] = {}

    for courses_wrapper in courses_wrappers:
        course_name_tag = courses_wrapper.find_previous('h3')
        course_name = course_name_tag.get_text(strip=True) if course_name_tag else "Unknown Course"

        all_dining_data[dining_hall_name][course_name] = {}

        for item in courses_wrapper.find_all('li'):
            item_name_element = item.find('div', class_='item-name')
            food_name = item_name_element.get_text(strip=True) if item_name_element else "N/A"

            nutrition_table = item.find('table', class_='nutrition-facts')
            if nutrition_table:
                serving_size, calories = extract_serving_size_and_calories(nutrition_table)
                nutrients = []

                for row in nutrition_table.find_all('tr')[1:]:
                    columns = row.find_all('td')
                    if len(columns) == 2:
                        nutrient_name, nutrient_value = split_nutrient_data(columns[0].get_text(strip=True))
                        nutrient_percentage = convert_percentage(columns[1].get_text(strip=True))
                        nutrients.append((nutrient_name, nutrient_percentage))

                all_dining_data[dining_hall_name][course_name][food_name] = {'serving_size': serving_size, 'calories': calories, 'nutrients': nutrients}

# Separate files for different meal types
breakfast_file = open("breakfast_options.txt", "w")
brunch_file = open("brunch_options.txt", "w")
lunch_file = open("lunch_options.txt", "w")
dinner_file = open("dinner_options.txt", "w")

for dining_hall, courses in all_dining_data.items():
    for course_name, course_items in courses.items():
        for food_name, data in course_items.items():
            course_data = f"Dining Hall: {dining_hall}\n"
            course_data += f"  Course: {course_name}\n"
            course_data += f"    Food Item: {food_name}\n"
            course_data += f"    Serving Size: {data['serving_size']}\n"
            course_data += f"    Calories: {data['calories']}\n"
            for nutrient_name, nutrient_value in data['nutrients'][1:]:  # Skip first nutrient
                if nutrient_name and nutrient_value:
                    course_data += f"      - {nutrient_name}: {nutrient_value}\n"

            # Determine the file to write based on the course name
            if "Breakfast" in course_name:
                breakfast_file.write(course_data)
            elif "Brunch" in course_name:
                brunch_file.write(course_data)
            elif "Lunch" in course_name:
                lunch_file.write(course_data)
            elif "Dinner" in course_name:
                dinner_file.write(course_data)

# Close all the files
breakfast_file.close()
brunch_file.close()
lunch_file.close()
dinner_file.close()

In [ ]:
import openai

file_path = 'dinner_options.txt'

with open(file_path, 'r') as file:
    data = file.read()

openai.api_key= #INSERT YOUR PERSONAL API KEY HERE
completion = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You're a nutritionist tasked with creating a tailored meal plan based on user data, a menu of 7 dining halls, and a meal course. Choose a dining hall and meal plan that matches the user's dietary needs and health goals. Meals must be from the same dining hall. Provide nutritional facts and explain your choices."},
    {"role": "user", "content": "The User is a male, 6'0, 200lbs, 20, health goal is to lose weight, no allergies, and no diet planned."},
    {"role": "user", "content": "The meal course is Dinner. Here is the menu: "}
  ]
)
print(completion.choices[0].message.content)